In [1]:
import pickle
import time
from selenium import webdriver

driver = webdriver.Chrome()

def guardar_cookies_facebook():
    driver.get("https://www.facebook.com")
    print("Por favor, inicia sesión manualmente en Facebook...")
    
    # Te damos 60 segundos para entrar y cerrar cualquier pop-up
    time.sleep(60) 
    
    with open("facebook_cookies.pkl", "wb") as file:
        pickle.dump(driver.get_cookies(), file)
    
    print("¡Cookies de Facebook guardadas!")
    driver.quit()

guardar_cookies_facebook()

Por favor, inicia sesión manualmente en Facebook...
¡Cookies de Facebook guardadas!


In [10]:
import pickle
import time
import random
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# --- CONFIGURACIÓN ---
def configurar_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    driver = webdriver.Chrome(options=chrome_options)
    return driver

driver = configurar_driver()

def esta_logueado():
    try:
        driver.find_element(By.XPATH, "//div[@role='navigation'] | //input[@placeholder='Buscar en Facebook']")
        return True
    except: return False

def cargar_cookies_fb():
    driver.get("https://www.facebook.com")
    time.sleep(4)
    try:
        with open("facebook_cookies.pkl", "rb") as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                if 'expiry' in cookie: del cookie['expiry']
                driver.add_cookie(cookie)
        driver.refresh()
        time.sleep(6)
        return esta_logueado()
    except: return False

def realizar_scraping_fb(tema):
    if not cargar_cookies_fb():
        return

    driver.get(f"https://www.facebook.com/search/posts/?q={tema}")
    print(f"Buscando: {tema}...")
    time.sleep(8)

    with open('comentarios_fb_reales.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Comentario"])

        for i in range(1, 11):
            try:
                print(f"\n--- Publicación #{i} ---")
                selector_post = f"//div[@aria-posinset='{i}']"
                post = driver.find_element(By.XPATH, selector_post)
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", post)
                time.sleep(2)

                # 1. Abrir Modal
                try:
                    boton_abrir = post.find_element(By.XPATH, ".//div[@role='button' and contains(., 'comentario')]")
                    driver.execute_script("arguments[0].click();", boton_abrir)
                    time.sleep(5)
                except: continue

                # 2. Cambiar a 'Todos los comentarios'
                try:
                    filtro_actual = driver.find_element(By.XPATH, "//div[@role='button']//span[contains(text(), 'relevantes') or contains(text(), 'Relevant')]")
                    driver.execute_script("arguments[0].click();", filtro_actual)
                    time.sleep(2)
                    opcion_todos = driver.find_element(By.XPATH, "//div[@role='menuitem']//span[contains(text(), 'Todos los comentarios') or contains(text(), 'All comments')]")
                    driver.execute_script("arguments[0].click();", opcion_todos)
                    time.sleep(5)
                except: pass

                # 3. Cargar comentarios (Scroll dentro del modal)
                print("   [+] Cargando comentarios...")
                for _ in range(10):
                    webdriver.ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
                    time.sleep(1.5)

                # 4. EXTRACCIÓN CON EL SELECTOR QUE ME PASASTE
                # Buscamos específicamente el div con style 'text-align: start' que contiene el texto
                bloques_texto = driver.find_elements(By.XPATH, "//div[@dir='auto' and @style='text-align: start;']")
                
                conteo_post = 0
                for bloque in bloques_texto:
                    try:
                        # Extraemos el texto plano (sin emojis ni etiquetas de nombres)
                        texto = bloque.text.strip()
                        
                        if len(texto) > 5:
                            # Limpieza para que el CSV no se rompa
                            fila = texto.replace('\n', ' ')
                            writer.writerow([fila])
                            conteo_post += 1
                            print(f"      [✔] Extraído: {fila[:50]}...")
                    except:
                        continue
                
                print(f"   [OK] {conteo_post} comentarios capturados del post {i}.")

                # 5. Cerrar y pasar al siguiente
                webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                time.sleep(2)

            except Exception as e:
                webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                continue

    print("\n✅ Proceso completado.")

if __name__ == "__main__":
    realizar_scraping_fb("Inteligencia Artificial")

Buscando: Inteligencia Artificial...

--- Publicación #1 ---
   [+] Cargando comentarios...
      [✔] Extraído: La Revolución de la Inteligencia Artificial:...
      [✔] Extraído: La inteligencia artificial (IA) ha revolucionado e...
      [✔] Extraído: #LecturaRecomendada...
      [✔] Extraído: Este libro trata sobre la inteligencia artificial,...
      [✔] Extraído: El ingenio (y la crítica) en redes sociales alcanz...
      [✔] Extraído: No cometas el error de usar #chatgpt  este Año 202...
      [✔] Extraído: La Revolución de la Inteligencia Artificial:...
      [✔] Extraído: La inteligencia artificial (IA) ha revolucionado e...
      [✔] Extraído: Ventajas Psicológicas de la IA...
      [✔] Extraído: -  Automatización de tareas repetitivas, liberando...
      [✔] Extraído: -  Personalización de experiencias, mejorando la s...
      [✔] Extraído: -  Análisis de grandes cantidades de datos para la...
      [✔] Extraído: Desventajas Psicológicas de la IA...
      [✔] Extraído: -  Pér